# Using Kaggle API to Download the data

In [2]:
# Link to GitHub


In [1]:
import pandas as pd
import opendatasets as od
import psycopg2

In [2]:
# Airbnb-property-data-from-texas
od.download("https://www.kaggle.com/datasets/PromptCloudHQ/airbnb-property-data-from-texas")

Skipping, found downloaded files in ".\airbnb-property-data-from-texas" (use force=True to force download)


In [3]:
# The data is in CSV format, read the data using pandas library
data = pd.read_csv('airbnb-property-data-from-texas/Airbnb_Texas_Rentals.csv')
data = data.drop(data.columns[[0,9]], axis=1)
print(data.shape)
print(data.dtypes)

(18259, 8)
average_rate_per_night     object
bedrooms_count             object
city                       object
date_of_listing            object
description                object
latitude                  float64
longitude                 float64
title                      object
dtype: object


In [4]:
# Visualize rows of the data
data.head(2)

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...


In [5]:
# create table airbnb_texas_rentals
# (
#     average_rate_per_night          varchar, 
#     bedrooms_count                  varchar,
#     city                            varchar,
#     date_of_listing                 varchar,
#     description                     varchar,
#     latitude                        float,
#     longitude                       float,
#     title                           varchar
# );
        

In [6]:
# Types of data that we have in our CSV file
data.dtypes

average_rate_per_night     object
bedrooms_count             object
city                       object
date_of_listing            object
description                object
latitude                  float64
longitude                 float64
title                      object
dtype: object

In [7]:
# In order to insert the data to PostgreSQL we have to adopt the format
replacements = {
    'object': 'varchar',
    'float64': 'float'
}

replacements

{'object': 'varchar', 'float64': 'float'}

In [8]:
# Now we created the string format that we need
col_str = ", ".join("{} {}".format(n,d) for (n, d) in zip(data.columns, data.dtypes.replace(replacements)))
col_str

'average_rate_per_night varchar, bedrooms_count varchar, city varchar, date_of_listing varchar, description varchar, latitude float, longitude float, title varchar'

# Python to PostgreSQL connection

In [9]:
# Uncomment if you need to install the library
#!pip install psycopg2

In [13]:
# Forming the connection to PostgreSQL
conn = psycopg2.connect(
    database="DataStorage",
    user="postgres",
    password="341741",
    host="localhost",
    port = "5433"
    )

cursor = conn.cursor()
print("sucessfull connection")

sucessfull connection


In [14]:
# Delete the table if it already exists
cursor.execute("drop table if exists airbnb_texas_rentals;")

In [15]:
# Create the table with columns
cursor.execute("create table airbnb_texas_rentals \
(average_rate_per_night varchar, bedrooms_count varchar, city varchar, date_of_listing varchar, \
description varchar, latitude float, longitude float, title varchar)")
conn.commit()

In [16]:
# Create .CSV from dataframe
data.to_csv('airbnb_texas_rentals.csv', header=data.columns, index=False, encoding='utf-8')
my_file = open('airbnb_texas_rentals.csv', 'r', encoding='utf-8')


In [17]:
# Inserting the file to db
SQL_STATEMENT = """
COPY airbnb_texas_rentals FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""
cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print("file copied to db")

file copied to db


In [18]:
cursor.execute("grant select on table airbnb_texas_rentals to public")
conn.commit()

print('table airbnb_texas_rentals imported to db')

table airbnb_texas_rentals imported to db


# Using some SQL query to explore the data that we stored

In [19]:
# SQL code as a string
sql = "SELECT * FROM airbnb_texas_rentals LIMIT 5;"

# Execute the SQL code
cursor.execute(sql)

# Fetch the results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()

('$27', '2', 'Humble', 'May 2016', 'Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\\n10min from IAH airport\\nAirport pick-up/drop off is available for $10/trip.', 30.0201379199512, -95.29399600425128, '2 Private rooms/bathroom 10min from IAH airport')
('$149', '4', 'San Antonio', 'November 2010', 'Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \\n\\nAmazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. \\n\\nFeaturing an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more\\n\\nEuropean inspired kitchen and “top of the line” decor. Driveway can park 4 cars.', 29.503067675606104, -98.4476879378504, 'Unique Location!